# Protein Sequence Extraction from Genomic DNA

**Goal:** Extract protein sequences for all 4 psilocybin BGC genes (PsiD, PsiK, PsiM, PsiH) from genomic DNA scaffolds

**Workflow:** Replicate the Colab workflow from `01_Colab_codes_InputFiles/Codes/`

1. ✅ Install tools (already done - conda environment `AncSeq`)
2. Check input files
3. Run exonerate (protein2genome) to extract CDS
4. Translate CDS to protein
5. Quality control
6. Save results

---

## Setup

In [ ]:
import os
import subprocess
from pathlib import Path
from Bio import SeqIO
from tqdm.notebook import tqdm
import time

# Set paths
PROJECT_DIR = Path.cwd().parent
INPUT_DIR = PROJECT_DIR / "01_Colab_codes_InputFiles" / "Files_paper_P_baeocystis"
EXAMPLE_OUTPUT = PROJECT_DIR / "02_ColabResults_Example"
RESULTS_DIR = Path.cwd() / "results"
RESULTS_DIR.mkdir(exist_ok=True)

print(f"Project directory: {PROJECT_DIR}")
print(f"Input directory: {INPUT_DIR}")
print(f"Results directory: {RESULTS_DIR}")
print(f"\nExample output directory: {EXAMPLE_OUTPUT}")

## Step 1: Check Input Files

Based on `02_check_files.txt`

In [ ]:
# List files in input directory
!ls -lh "$INPUT_DIR"

# Expected files:
# 1. Reference protein: PsiD_Psilocybe_cubensis_reference.faa
# 2. Genome scaffolds: Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta
# 3. (Optional) GFF from paper: Psilocybe_baeocystis_WTU-F-011245-redundands.gff

In [ ]:
# Check reference protein file
reference_protein = INPUT_DIR / "PsiD_Psilocybe_cubensis_reference.faa"
print(f"Reference protein exists: {reference_protein.exists()}")

if reference_protein.exists():
    for record in SeqIO.parse(reference_protein, "fasta"):
        print(f"\nID: {record.id}")
        print(f"Description: {record.description}")
        print(f"Length: {len(record.seq)} aa")
        print(f"Sequence preview: {str(record.seq)[:70]}...")

In [ ]:
# Check genome scaffold file
genome_scaffolds = INPUT_DIR / "Psilocybe_baeocystis_WTU-F-011245.scaffolds.fasta"
print(f"Genome scaffolds exist: {genome_scaffolds.exists()}")

if genome_scaffolds.exists():
    # Count scaffolds
    scaffold_count = 0
    total_length = 0
    for record in SeqIO.parse(genome_scaffolds, "fasta"):
        scaffold_count += 1
        total_length += len(record.seq)
    
    print(f"Number of scaffolds: {scaffold_count}")
    print(f"Total genome length: {total_length:,} bp")
    print(f"Average scaffold size: {total_length // scaffold_count:,} bp")

## Step 2: Run Exonerate (Extract CDS)

Based on `03_exonerate_best_hit.txt`

This extracts the coding sequence (CDS) by aligning the reference protein to the genome scaffolds.

In [ ]:
# Define output files
output_cds = RESULTS_DIR / "P_baeocystis_PsiD.cds.fa"
output_gff = RESULTS_DIR / "P_baeocystis_PsiD.gff"

# Command 1: Extract CDS
cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">PsiD|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''

print("Running exonerate to extract CDS...")
print(f"Command: {cmd_cds}\n")

# Run command
result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ CDS extraction complete!")
    print(f"  Output: {output_cds}")
else:
    print(f"✗ Error: {result.stderr}")

In [ ]:
# Command 2: Extract GFF annotation
cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''

print("Running exonerate to extract GFF...")
print(f"Command: {cmd_gff}\n")

# Run command
result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ GFF extraction complete!")
    print(f"  Output: {output_gff}")
else:
    print(f"✗ Error: {result.stderr}")

In [ ]:
# Check the extracted CDS
print("Extracted CDS:")
if output_cds.exists():
    for record in SeqIO.parse(output_cds, "fasta"):
        print(f"  ID: {record.id}")
        print(f"  Length: {len(record.seq)} bp")
        print(f"  Sequence preview: {str(record.seq)[:70]}...")
else:
    print("  CDS file not found!")

## Step 3: Translate CDS to Protein

Based on `04_translate.txt`

Use EMBOSS transeq to translate the CDS to protein sequence.

In [ ]:
# Define output protein file
output_protein = RESULTS_DIR / "P_baeocystis_PsiD.prot.fa"

# Command: Translate CDS to protein (frame 1)
cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'

print("Translating CDS to protein...")
print(f"Command: {cmd_translate}\n")

# Run command
result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Translation complete!")
    print(f"  Output: {output_protein}")
else:
    print(f"✗ Error: {result.stderr}")

In [ ]:
# Check the translated protein
print("Translated Protein:")
if output_protein.exists():
    for record in SeqIO.parse(output_protein, "fasta"):
        print(f"  ID: {record.id}")
        print(f"  Length: {len(record.seq)} aa")
        print(f"  Sequence: {str(record.seq)}")
else:
    print("  Protein file not found!")

## Step 4: Quality Control

Based on `05_quality_control.txt`

Check for:
1. Internal stop codons (should be none)
2. Protein length

In [ ]:
# Check for internal stop codons
print("=" * 60)
print("QUALITY CONTROL")
print("=" * 60)

if output_protein.exists():
    for record in SeqIO.parse(output_protein, "fasta"):
        sequence = str(record.seq)
        
        # Check for internal stops (excluding terminal *)
        internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
        stop_count = internal_seq.count('*')
        
        if stop_count == 0:
            print("✅ No internal stop codons found")
        else:
            print(f"⚠️  WARNING: {stop_count} internal stop codon(s) found!")
            # Show positions
            for i, aa in enumerate(internal_seq):
                if aa == '*':
                    print(f"    Position {i+1}: STOP")
        
        # Check protein length
        protein_length = len(sequence)
        print(f"\nProtein length: {protein_length} aa")
        
        # Compare to reference
        for ref_record in SeqIO.parse(reference_protein, "fasta"):
            ref_length = len(ref_record.seq)
            print(f"Reference length: {ref_length} aa")
            
            diff = protein_length - ref_length
            if diff == 0:
                print("✅ Length matches reference exactly")
            else:
                print(f"⚠️  Length difference: {diff:+d} aa ({abs(diff/ref_length*100):.1f}%)")

In [ ]:
# Preview the first few lines of the protein file
print("\nProtein FASTA file preview:")
!head -n 5 "{output_protein}"

## Step 5: Compare with Example Output

Verify our results match the example from `02_ColabResults_Example/`

In [ ]:
# Compare ALL output files with examples
print("Comparing with example output:")
print("=" * 60)

# 1. Compare PROTEIN sequences
example_protein = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.prot.fa"
print("\n1. PROTEIN COMPARISON:")
if example_protein.exists() and output_protein.exists():
    our_seq = str(list(SeqIO.parse(output_protein, "fasta"))[0].seq)
    example_seq = str(list(SeqIO.parse(example_protein, "fasta"))[0].seq)
    
    print(f"   Our length: {len(our_seq)} aa")
    print(f"   Example length: {len(example_seq)} aa")
    
    if our_seq == example_seq:
        print("   ✅ PROTEIN: Perfect match!")
    else:
        print("   ⚠️  PROTEIN: Sequences differ")
        for i, (aa1, aa2) in enumerate(zip(our_seq, example_seq)):
            if aa1 != aa2:
                print(f"      Position {i+1}: {aa1} vs {aa2}")

# 2. Compare CDS sequences
example_cds = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.cds.fa"
print("\n2. CDS COMPARISON:")
if example_cds.exists() and output_cds.exists():
    our_cds = str(list(SeqIO.parse(output_cds, "fasta"))[0].seq)
    example_cds_seq = str(list(SeqIO.parse(example_cds, "fasta"))[0].seq)
    
    print(f"   Our length: {len(our_cds)} bp")
    print(f"   Example length: {len(example_cds_seq)} bp")
    
    if our_cds == example_cds_seq:
        print("   ✅ CDS: Perfect match!")
    else:
        print(f"   ⚠️  CDS: Sequences differ ({sum(1 for a,b in zip(our_cds, example_cds_seq) if a!=b)} differences)")

# 3. Compare GFF files (just check if they exist and basic structure)
example_gff = EXAMPLE_OUTPUT / "P_baeocystis_PsiD.gff"
print("\n3. GFF COMPARISON:")
if example_gff.exists() and output_gff.exists():
    our_gff_lines = open(output_gff).readlines()
    example_gff_lines = open(example_gff).readlines()
    
    print(f"   Our GFF lines: {len(our_gff_lines)}")
    print(f"   Example GFF lines: {len(example_gff_lines)}")
    
    if our_gff_lines == example_gff_lines:
        print("   ✅ GFF: Perfect match!")
    else:
        print(f"   ⚠️  GFF: Files differ")

print("\n" + "=" * 60)
print("VERIFICATION COMPLETE")

## Summary

**Output files created:**
- CDS (DNA): `results/P_baeocystis_PsiD.cds.fa`
- Protein: `results/P_baeocystis_PsiD.prot.fa`
- GFF annotation: `results/P_baeocystis_PsiD.gff`

**Next steps:**
1. ✅ Verify this works for PsiD
2. Replicate for other genes (PsiK, PsiM, PsiH)
3. Batch process all species from `03_Sequences_Paper/`
4. Combine all extracted proteins for phylogenetic analysis

In [ ]:
# If GFF differs, show the differences
if example_gff.exists() and output_gff.exists():
    our_gff_lines = open(output_gff).readlines()
    example_gff_lines = open(example_gff).readlines()
    
    if our_gff_lines != example_gff_lines:
        print("\nGFF Differences (first 5 differences shown):")
        print("-" * 60)
        diff_count = 0
        for i, (our_line, ex_line) in enumerate(zip(our_gff_lines, example_gff_lines)):
            if our_line != ex_line and diff_count < 5:
                print(f"\nLine {i+1}:")
                print(f"  Ours:    {our_line.strip()}")
                print(f"  Example: {ex_line.strip()}")
                diff_count += 1

---

# Extract All 4 BGC Enzymes

Now that we've validated the workflow with PsiD, let's extract all 4 enzymes from P. baeocystis:
1. **PsiD** - Tryptophan decarboxylase
2. **PsiK** - Kinase
3. **PsiM** - Methyltransferase
4. **PsiH** - P450 monooxygenase

---

In [ ]:
# Define all 4 BGC genes with their reference proteins
REF_PROTEIN_DIR = Path.cwd() / "reference_proteins"

BGC_GENES = {
    'PsiD': {
        'name': 'Tryptophan decarboxylase',
        'reference': INPUT_DIR / "PsiD_Psilocybe_cubensis_reference.faa",
        'function': 'Converts tryptophan to tryptamine'
    },
    'PsiK': {
        'name': 'Kinase',
        'reference': REF_PROTEIN_DIR / "PsiK_Psilocybe_cubensis_reference.faa",
        'function': 'Phosphorylates intermediate'
    },
    'PsiM': {
        'name': 'Methyltransferase',
        'reference': REF_PROTEIN_DIR / "PsiM_Psilocybe_cubensis_reference.faa",
        'function': 'Methylates tryptamine'
    },
    'PsiH': {
        'name': 'P450 monooxygenase',
        'reference': REF_PROTEIN_DIR / "PsiH_Psilocybe_cubensis_reference.faa",
        'function': 'Hydroxylates at C-4 position (critical for activity)'
    }
}

# Verify all reference files exist
print("Checking reference protein files:")
for gene, info in BGC_GENES.items():
    exists = info['reference'].exists()
    symbol = "✅" if exists else "❌"
    print(f"  {symbol} {gene}: {info['reference'].name}")

## PsiD - Tryptophan Decarboxylase

Extract PsiD protein from P. baeocystis genome

In [ ]:
gene = 'PsiD'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

## PsiK - Kinase

Extract PsiK protein from P. baeocystis genome

In [ ]:
gene = 'PsiK'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

## PsiM - Methyltransferase

Extract PsiM protein from P. baeocystis genome

In [ ]:
gene = 'PsiM'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

## PsiH - P450 Monooxygenase

Extract PsiH protein from P. baeocystis genome

In [ ]:
gene = 'PsiH'
reference_protein = BGC_GENES[gene]['reference']

# Define output files
output_cds = RESULTS_DIR / f"P_baeocystis_{gene}.cds.fa"
output_gff = RESULTS_DIR / f"P_baeocystis_{gene}.gff"
output_protein = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"

print(f"{'='*60}")
print(f"Processing {gene} - {BGC_GENES[gene]['name']}")
print(f"{'='*60}\n")

# Create progress bar for 4 steps
with tqdm(total=4, desc=f"{gene}", bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}]') as pbar:
    
    # Step 1: Extract CDS
    pbar.set_description(f"{gene}: Extracting CDS")
    cmd_cds = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --ryo ">{gene}|%ti:%tab-%tae(%tS)\\n%tcs\\n" > "{output_cds}"'''
    
    result = subprocess.run(cmd_cds, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ CDS extracted: {output_cds.name}")
    else:
        print(f"   ✗ Error: {result.stderr}")
    pbar.update(1)
    
    # Step 2: Extract GFF
    pbar.set_description(f"{gene}: Extracting GFF")
    cmd_gff = f'''exonerate --model protein2genome \
  "{reference_protein}" \
  "{genome_scaffolds}" \
  --bestn 1 \
  --showalignment no --showvulgar no --verbose 0 \
  --showtargetgff yes > "{output_gff}"'''
    
    result = subprocess.run(cmd_gff, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ GFF extracted: {output_gff.name}")
    pbar.update(1)
    
    # Step 3: Translate to protein
    pbar.set_description(f"{gene}: Translating")
    cmd_translate = f'transeq -sequence "{output_cds}" -outseq "{output_protein}" -frame 1'
    result = subprocess.run(cmd_translate, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✓ Protein translated: {output_protein.name}")
    pbar.update(1)
    
    # Step 4: Quality Control
    pbar.set_description(f"{gene}: Quality check")
    if output_protein.exists():
        for record in SeqIO.parse(output_protein, "fasta"):
            sequence = str(record.seq)
            internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
            stop_count = internal_seq.count('*')
            
            print(f"\n   Quality Control:")
            if stop_count == 0:
                print(f"   ✅ No internal stop codons")
            else:
                print(f"   ⚠️  {stop_count} internal stop codon(s) found!")
            
            protein_length = len(sequence)
            for ref_record in SeqIO.parse(reference_protein, "fasta"):
                ref_length = len(ref_record.seq)
                diff = protein_length - ref_length
                if diff == 0:
                    print(f"   ✅ Length: {protein_length} aa (matches reference)")
                else:
                    print(f"   ⚠️  Length: {protein_length} aa (ref: {ref_length} aa, diff: {diff:+d})")
    pbar.update(1)
    pbar.set_description(f"{gene}: Complete")

print(f"\n{'='*60}\n")

## Final Summary

Review all extracted proteins

In [ ]:
import pandas as pd

# Collect results for all 4 genes
results = []

for gene in ['PsiD', 'PsiK', 'PsiM', 'PsiH']:
    protein_file = RESULTS_DIR / f"P_baeocystis_{gene}.prot.fa"
    
    if protein_file.exists():
        record = list(SeqIO.parse(protein_file, "fasta"))[0]
        sequence = str(record.seq)
        internal_seq = sequence[:-1] if sequence.endswith('*') else sequence
        
        # Get reference length
        ref_file = BGC_GENES[gene]['reference']
        ref_record = list(SeqIO.parse(ref_file, "fasta"))[0]
        
        results.append({
            'Gene': gene,
            'Enzyme': BGC_GENES[gene]['name'],
            'Length (aa)': len(sequence),
            'Reference (aa)': len(ref_record.seq),
            'Difference': len(sequence) - len(ref_record.seq),
            'Stop codons': internal_seq.count('*'),
            'Status': '✅' if internal_seq.count('*') == 0 else '⚠️'
        })

# Display as table
df = pd.DataFrame(results)
print("\n" + "="*80)
print("PROTEIN EXTRACTION SUMMARY - Psilocybe baeocystis")
print("="*80 + "\n")
print(df.to_string(index=False))
print("\n" + "="*80)
print(f"\nTotal proteins extracted: {len(results)}/4")
print(f"Results saved in: {RESULTS_DIR}")
print("="*80)